## 1) Load Raw Data from Unity Catalog Volume

In this section, we:
1. Define a **schema** for the raw churn dataset to ensure data type consistency
2. Read the CSV file from a **Unity Catalog Volume** (external storage location)
3. Load the data into a Spark DataFrame for inspection

### Why Define a Schema?

- **Data Quality**: Enforces expected data types (prevents strings being read as integers, etc.)
- **Performance**: Spark doesn't need to infer schema by scanning the entire file
- **Clarity**: Acts as documentation for the dataset structure

### Dataset Overview

This telecom customer churn dataset contains:
- **Customer demographics**: gender, senior citizen status, partner, dependents
- **Account information**: tenure (months with company), contract type, payment method
- **Service subscriptions**: phone service, internet type, streaming services, tech support
- **Financial metrics**: monthly charges, total charges
- **Target variable**: `churn` (whether the customer left the company)

The raw data is stored in a **Unity Catalog Volume** at `/Volumes/source_system/source/data_volume`, which allows us to manage files within Unity Catalog's governance framework.

In [0]:
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    DoubleType
)

churn_schema = StructType([
    StructField("customer_id",        StringType(),  False),
    StructField("gender",             StringType(),  False),
    StructField("senior_citizen",     IntegerType(), False),
    StructField("partner",            StringType(),  False),
    StructField("dependents",         StringType(),  False),
    StructField("tenure",             IntegerType(), False),
    StructField("phone_service",      StringType(),  False),
    StructField("multiple_lines",     StringType(),  False),
    StructField("internet_service",   StringType(),  False),
    StructField("online_security",    StringType(),  False),
    StructField("online_backup",      StringType(),  False),
    StructField("device_protection",  StringType(),  False),
    StructField("tech_support",       StringType(),  False),
    StructField("streaming_tv",       StringType(),  False),
    StructField("streaming_movies",   StringType(),  False),
    StructField("contract",           StringType(),  False),
    StructField("paperless_billing",  StringType(),  False),
    StructField("payment_method",     StringType(),  False),
    StructField("monthly_charges",    DoubleType(),  False),
    StructField("total_charges",      DoubleType(),  False),
    StructField("churn",              StringType(),  False)
])


In [0]:
churn_raw_df = (
    spark.read
         .option("header", "true")
         .option("delimiter", ";")
         .schema(churn_schema)
         .csv("/Volumes/source_system/source/data_volume")
)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-6144026142441633>, line 5
      1 churn_raw_df = (
      2     spark.read
      3          .option("header", "true")
      4          .option("delimiter", ";")
----> 5          .schema(churn_schema)
      6          .csv("/Volumes/source_system/source/data_volume")
      7 )

NameError: name 'churn_schema' is not defined

In [0]:
display(churn_raw_df)

customer_id gender senior_citizen partner dependents tenure phone_service multiple_lines internet_service online_security online_backup device_protection tech_support streaming_tv streaming_movies contract paperless_billing payment_method monthly_charges total_charges churn 1329-VHWNP Female 0 No No 7 No No phone service DSL No No No No No No Month-to-month No Bank transfer (automatic) 25.05 189.95 No 2984-MIIZL Male 0 No No 4 Yes No Fiber optic No No Yes No No No Month-to-month Yes Bank transfer (automatic) 74.8 321.9 Yes 0266-GMEAO Male 0 Yes Yes 72 Yes Yes Fiber optic Yes Yes Yes Yes Yes Yes Two year Yes Credit card (automatic) 114.3 8058.55 No 5590-YRFJT Female 0 Yes No 20 No No phone service DSL No No No No No No Month-to-month Yes Electronic check 24.45 482.8 Yes 5574-NXZIU Male 0 No No 63 Yes No Fiber optic Yes Yes Yes Yes Yes Yes Two year No Credit card (automatic) 109.2 7049.75 No 0019-GFNTW Female 0 No No 56 No No phone service DSL Yes Yes Yes Yes No No Two year No Bank transfer (automatic) 45.05 2560.1 No 4256-ZWTZI Female 0 No Yes 5 Yes No DSL No No No Yes No No Month-to-month Yes Mailed check 51.0 286.8 No 8309-PPCED Female 0 Yes No 72 Yes Yes Fiber optic No Yes Yes Yes Yes Yes Two year Yes Bank transfer (automatic) 110.45 7982.5 No 4098-NAUKP Male 1 Yes Yes 68 Yes Yes Fiber optic No Yes Yes No No No Month-to-month Yes Electronic check 84.65 5683.6 No 5196-WPYOW Male 0 Yes Yes 67 Yes No DSL Yes Yes No Yes No No One year No Mailed check 60.05 3994.05 No 4608-LCIMN Male 0 Yes Yes 8 Yes No DSL No No No No No No Month-to-month No Bank transfer (automatic) 44.65 369.15 No 1485-YDHMM Male 0 Yes Yes 52 Yes Yes Fiber optic No Yes No Yes No Yes Month-to-month Yes Bank transfer (automatic) 93.25 4631.7 No 4054-CUMIA Female 0 Yes Yes 18 Yes No No No internet service No internet service No internet service No internet service No internet service No internet service Month-to-month Yes Mailed check 20.25 401.95 No 0603-TPMIB Female 0 Yes Yes 59 Yes Yes No No internet service No internet service No internet service No internet service No internet service No internet service Two year No Mailed check 25.45 1534.05 No 2525-GVKQU Female 0 No No 60 Yes No No No internet service No internet service No internet service No internet service No internet service No internet service Two year No Mailed check 20.6 1093.0 No 8161-QYMTT Male 0 No No 7 Yes Yes Fiber optic No No No No Yes Yes Month-to-month Yes Electronic check 94.1 701.3 No 9581-GVBXT Male 0 Yes Yes 59 No No phone service DSL Yes No Yes No No No One year No Mailed check 34.8 1980.3 No 5862-BRIXZ Male 0 No No 46 No No phone service DSL Yes No Yes Yes Yes Yes Two year Yes Bank transfer (automatic) 60.75 2893.4 No 5404-GGUKR Male 0 No No 5 Yes No DSL No Yes No No No No Month-to-month Yes Electronic check 51.35 262.3 No 3308-JSGML Male 1 Yes No 59 No No phone service DSL Yes No Yes Yes Yes Yes Two year Yes Credit card (automatic) 64.05 3886.85 No 2126-GSEGL Female 0 Yes No 70 Yes Yes DSL No Yes Yes Yes Yes Yes Two year Yes Bank transfer (automatic) 84.8 5917.55 No 3677-TNKIO Female 0 No No 14 Yes No Fiber optic No No No No No No Month-to-month Yes Credit card (automatic) 71.0 914.0 Yes 0440-QEXBZ Female 0 No No 44 Yes Yes DSL No No No No No No Month-to-month Yes Mailed check 50.15 2139.1 No 2434-EEVDB Female 0 Yes No 64 Yes Yes Fiber optic Yes No Yes No Yes No Two year Yes Credit card (automatic) 94.6 5948.7 No 6762-QVYJO Female 0 Yes Yes 58 No No phone service DSL Yes No Yes Yes Yes Yes Two year No Mailed check 59.75 3624.35 No 6199-IWKGC Female 1 Yes No 46 Yes Yes Fiber optic No No No Yes Yes Yes One year No Electronic check 100.25 4753.85 No 2675-DHUTR Male 1 Yes No 58 Yes Yes Fiber optic No Yes No No Yes Yes Month-to-month Yes Electronic check 98.9 5780.7 No 8152-VETUR Female 0 Yes No 72 Yes Yes Fiber optic No Yes No No Yes Yes Two year Yes Credit card (automatic) 97.7 6869.7 No 9667-EQRXU Female 1 No No 30 No No phone service DSL No No Yes No Yes No Month-to-month Yes Electroni

## 2) Create Unity Catalog Structure and Save to Bronze Layer

Unity Catalog organizes data using a **3-level namespace**:
```
catalog.schema.table
```

### Creating the Medallion Architecture

We create a **medallion architecture** for data processing:

| Layer | Purpose | Table Example |
|-------|---------|---------------|
| **Bronze** | Raw, unprocessed data as ingested | `churn_lab.bronze.churn_raw` |
| **Silver** | Cleaned, validated, deduplicated data | `churn_lab.silver.churn_cleaned` |
| **Gold** | Business-level aggregates and ML features | `churn_lab.gold.churn_features` |

### What This Code Does:

1. **Creates the catalog and schemas** (if they don't already exist)
   - Catalog: `churn_lab`
   - Schemas: `bronze`, `silver`, `gold`

2. **Writes raw data to Bronze layer** as a Delta table
   - Format: **Delta Lake** (ACID transactions, time travel, schema enforcement)
   - Mode: **Overwrite** (replaces existing data - use `append` for incremental loads)

3. **Verifies the data load** by counting rows

4. **Queries the Bronze table** to inspect customers with DSL internet service

### Why Bronze Layer?

The Bronze layer preserves the **raw, unchanged data** exactly as it was received. This allows us to:
- Re-process data if downstream transformations have bugs
- Audit the original source data
- Maintain a historical archive

Next, we'll clean and transform this data into the **Silver layer**.

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS churn_lab;

CREATE SCHEMA IF NOT EXISTS churn_lab.bronze;
CREATE SCHEMA IF NOT EXISTS churn_lab.silver;
CREATE SCHEMA IF NOT EXISTS churn_lab.gold;

In [0]:
bronze_table = "churn_lab.bronze.churn_raw"

(churn_raw_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(bronze_table)
)

spark.table(bronze_table).count()

7043

In [0]:
%sql 
select * from churn_lab.bronze.churn_raw where internet_service = "DSL" limit 10

customer_id gender senior_citizen partner dependents tenure phone_service multiple_lines internet_service online_security online_backup device_protection tech_support streaming_tv streaming_movies contract paperless_billing payment_method monthly_charges total_charges churn 1329-VHWNP Female 0 No No 7 No No phone service DSL No No No No No No Month-to-month No Bank transfer (automatic) 25.05 189.95 No 5590-YRFJT Female 0 Yes No 20 No No phone service DSL No No No No No No Month-to-month Yes Electronic check 24.45 482.8 Yes 0019-GFNTW Female 0 No No 56 No No phone service DSL Yes Yes Yes Yes No No Two year No Bank transfer (automatic) 45.05 2560.1 No 4256-ZWTZI Female 0 No Yes 5 Yes No DSL No No No Yes No No Month-to-month Yes Mailed check 51.0 286.8 No 5196-WPYOW Male 0 Yes Yes 67 Yes No DSL Yes Yes No Yes No No One year No Mailed check 60.05 3994.05 No 4608-LCIMN Male 0 Yes Yes 8 Yes No DSL No No No No No No Month-to-month No Bank transfer (automatic) 44.65 369.15 No 9581-GVBXT Male 0 Yes Yes 59 No No phone service DSL Yes No Yes No No No One year No Mailed check 34.8 1980.3 No 5862-BRIXZ Male 0 No No 46 No No phone service DSL Yes No Yes Yes Yes Yes Two year Yes Bank transfer (automatic) 60.75 2893.4 No 5404-GGUKR Male 0 No No 5 Yes No DSL No Yes No No No No Month-to-month Yes Electronic check 51.35 262.3 No 3308-JSGML Male 1 Yes No 59 No No phone service DSL Yes No Yes Yes Yes Yes Two year Yes Credit card (automatic) 64.05 3886.85 No